In [ ]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [ ]:
from lib import *

In [ ]:
wdir = "/media/windows/projects/bias_correction/data/meteo/cmcc_cm/"

In [ ]:
def delete_first_row(file_path_name):
    out_path_name = file_path_name[:-4] + "_temporary_file.csv"
    # print(out_path_name)
    # command = '''sed -i '1d' {file_path_name}'''
    command = '''tail -n +2 {file_path_name} > {out_path_name}'''
    p = subprocess.Popen(command.format(file_path_name=file_path_name, out_path_name=out_path_name),
                         shell=True, stdout=subprocess.PIPE)
    
    return out_path_name

In [ ]:
list_of_outputs = []
output_path = "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/"
mkNestedDir(output_path)

In [ ]:
# CMCC data
input_paths = [
    wdir + "precipitation_model/pr_hist_entire_period/output/",
    wdir + "precipitation_model/pr_project/output/",
    wdir + "temperature_model/tas_hist_entire_period/output/",
    wdir + "temperature_model/tas_project/output/"
]

for input_path in input_paths:

    station_dirs = glob.glob(input_path + "*/")

    all_data = pd.DataFrame()

    for station_dir in station_dirs:
        # print("Opening: " + station_dir)
        to_aggregate_files = glob.glob(station_dir + "*.csv")

        station_id = station_dir.split("/")[-2]
        # print("station_id: " + station_id)
        var = station_dir.split("/")[-4].split("_")[0]
        # period = station_dir.split("/")[-4].split("_")[1]

        tmp_aggr_data = pd.DataFrame()

        for f in to_aggregate_files:

            if "_temporary_file" in f:
                try:
                    os.remove(f)
                    print("Removed: " + f)
                    continue
                except:
                    continue

            print(f)

            f = delete_first_row(f)
            time.sleep(3)
            tmp_data = pd.read_csv(f, skiprows=0, header=None)
            tmp_data['date'] = pd.to_datetime(tmp_data[0], format="%m/%Y")
            tmp_data.drop(columns=[0], inplace=True)
            tmp_data.set_index('date', inplace=True)
            os.remove(f)

            # tmp_data = tmp_data.rename(columns={1:"values"})

            tmp_aggr_data = pd.concat([tmp_aggr_data, tmp_data]).sort_index()

        # tmp_aggr_data.index = [pd.to_datetime(dt.datetime.strftime(t, "%Y-%m-%d %H:%M:%S"), format="%Y-%m-%d %H:%M:%S")
        #                        for t in tmp_aggr_data.index]

        try:
            all_data[station_id] = tmp_aggr_data
        except:
            continue

        del [station_id]

    all_data.index = [dt.datetime.strftime(
        t, "%Y-%m-%d %H:%M:%S") for t in all_data.index]
    all_data.index.name = 'datetime'
    # all_data.to_csv( input_path + "{var}_{period}.csv".format(var=var, period=period) )
    output_path_name = output_path + station_dir.split("/")[-4] + ".csv"
    all_data.to_csv( output_path_name )
    list_of_outputs.append( output_path_name )
    del [output_path_name]


In [ ]:
### observed data
input_path = wdir + "observations/obs/"

pr_data = pd.DataFrame()
tas_data = pd.DataFrame()

station_files = glob.glob( input_path + "*.xlsx" )

for station_file in station_files:

    tmp_file = pd.read_excel(station_file,engine='openpyxl')
    
    try:
        tmp_file = pd.read_excel(station_file,engine='openpyxl')
    except:
        print("Skipping.." + e)
        continue

    tmp_data = tmp_file[['PRCP (mm)','TAVG (°C)']]
    tmp_data.index = [ pd.to_datetime(d, format="%Y-%m") for d in tmp_file['DATE'] ]

    del [tmp_file]

    station_id = station_file.split("/")[-1].split("_")[0]
    
    if station_id == 'TX':
        station_id = station_file.split("/")[-1].split("_")[0] + '_' + station_file.split("/")[-1].split("_")[1]
    # print("station_id: " + station_id)

    pr_data[station_id] = tmp_data[['PRCP (mm)']]
    tas_data[station_id] = tmp_data[['TAVG (°C)']]
    
pr_data.index = [dt.datetime.strftime(t, "%Y-%m-%d %H:%M:%S") for t in pr_data.index]
pr_data.index.name = 'datetime'
output_path_name = output_path + "{var}_{period}.csv".format(var='pr', period='obs')
pr_data.to_csv( output_path_name )
list_of_outputs.append( output_path_name )
del [output_path_name]

tas_data.index = [dt.datetime.strftime(t, "%Y-%m-%d %H:%M:%S") for t in tas_data.index]
tas_data.index.name = 'datetime'
output_path_name = output_path + "{var}_{period}.csv".format(var='tas', period='obs')
tas_data.to_csv( output_path_name )
list_of_outputs.append( output_path_name )
del [output_path_name]

In [ ]:
# HISTORICAL CASE STUDY
# 1970-1995 calibration + 1996-2005 validation == 1970-2005 timeseries

metadata_01 = {
    "testcase": "testcase_01",
    "variables": [{
        "variable": "precipitation",
        "calibration": {"start_datetime": "1970/01/01 00:00:00",
                        "end_datetime": "1995/12/31 23:00:00",
                        "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_hist_entire_period.csv"},
        "validation": {"start_datetime": "1996/01/01 00:00:00",
                       "end_datetime": "2005/12/31 23:00:00",
                       "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_hist_entire_period.csv"},
        "reference": {"start_datetime": "1970/01/01 00:00:00",
                      "end_datetime": "2005/12/31 23:00:00",
                      "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_obs.csv"},
        "output_name": {"model": "pr_hist_1970_2005.csv", "reference": "pr_obs_1970_2005.csv"}},
        {"variable": "temperature",
         "calibration": {"start_datetime": "1970/01/01 00:00:00",
                         "end_datetime": "1995/12/31 23:00:00",
                         "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_hist_entire_period.csv"},
         "validation": {"start_datetime": "1996/01/01 00:00:00",
                        "end_datetime": "2005/12/31 23:00:00",
                        "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_hist_entire_period.csv"},
         "reference": {"start_datetime": "1970/01/01 00:00:00",
                       "end_datetime": "2005/12/31 23:00:00",
                       "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_obs.csv"},
         "output_name": {"model": "tas_hist_1970_2005.csv", "reference": "tas_obs_1970_2005.csv"}}
    ],
    "output_path": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/"
}

# HISTORICAL+PROJECTION CASE STUDY
# 1970-2005 calibration + 2006-2050 validation == 1970-2005 historical timeseries + 2006-2050 project timeseries

metadata_02 = {
    "testcase": "testcase_02",
    "variables": [
        {"variable": "precipitation",
         "calibration": {"start_datetime": "1970/01/01 00:00:00",
                         "end_datetime": "2005/12/31 23:00:00",
                         "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_hist_entire_period.csv"},
         "validation": {"start_datetime": "2006/01/01 00:00:00",
                        "end_datetime": "2050/12/31 23:00:00",
                        "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_project.csv"},
         "reference": {"start_datetime": "1970/01/01 00:00:00",
                       "end_datetime": "2005/12/31 23:00:00",
                       "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/pr_obs.csv"},
         "output_name": {"model": "pr_hist_proj_1970_2050.csv", "reference": "pr_obs_1970_2005.csv"}},
        {"variable": "temperature",
         "calibration": {"start_datetime": "1970/01/01 00:00:00",
                         "end_datetime": "2005/12/31 23:00:00",
                         "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_hist_entire_period.csv"},
         "validation": {"start_datetime": "1996/01/01 00:00:00",
                        "end_datetime": "2050/12/31 23:00:00",
                        "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_project.csv"},
         "reference": {"start_datetime": "1970/01/01 00:00:00",
                       "end_datetime": "2005/12/31 23:00:00",
                       "data": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/tas_obs.csv"},
         "output_name": {"model": "tas_hist_proj_1970_2050.csv", "reference": "tas_obs_1970_2005.csv"}
         }
    ],
    "output_path": "/media/windows/projects/bias_correction/applications/cmcc_cm/data/pre_processed_202206281324/"
}

data_to_extract = [metadata_01, metadata_02]


In [ ]:
for meta in data_to_extract:

    curr_output_path = meta['output_path'] + meta['testcase']
    mkNestedDir(curr_output_path)

    for var in meta['variables']:
        
        start_cal = pd.to_datetime( var['calibration']['start_datetime'] )
        end_cal = pd.to_datetime( var['calibration']['end_datetime'] )

        curr_data_cal = pd.read_csv( var['calibration']['data'] )
        curr_data_cal['datetime'] = pd.to_datetime( curr_data_cal['datetime'], format="%Y-%m-%d %H:%M:%S" )
        curr_data_cal.set_index('datetime', inplace=True)
        curr_data_cal = curr_data_cal[start_cal:end_cal]

        start_val = pd.to_datetime( var['validation']['start_datetime'] )
        end_val = pd.to_datetime( var['validation']['end_datetime'] )

        curr_data_val = pd.read_csv( var['validation']['data'] )
        curr_data_val['datetime'] = pd.to_datetime( curr_data_val['datetime'], format="%Y-%m-%d %H:%M:%S" )
        curr_data_val.set_index('datetime', inplace=True)
        curr_data_val = curr_data_val[start_val:end_val]

        # merge the calibration and validation period
        curr_data_model = pd.concat([ curr_data_cal,curr_data_val ])
        
        curr_data_model = round( curr_data_model, 2 )

        # save the data
        curr_data_model.index = [ dt.datetime.strftime(d,format="%Y-%m-%d %H:%M:%S") for d in curr_data_model.index ]
        curr_data_model.index.name = "datetime"
        curr_data_model.to_csv( curr_output_path + "/" + var['output_name']['model'] )

        del [curr_data_model]

        start_ref = pd.to_datetime( var['reference']['start_datetime'] )
        end_ref = pd.to_datetime( var['reference']['end_datetime'] )

        curr_data_ref = pd.read_csv( var['reference']['data'] )
        curr_data_ref['datetime'] = pd.to_datetime( curr_data_ref['datetime'], format="%Y-%m-%d %H:%M:%S" )
        curr_data_ref.set_index('datetime', inplace=True)
        curr_data_ref = curr_data_ref[start_ref:end_ref]

        curr_data_ref = round( curr_data_ref, 2 )

        # save the data
        curr_data_ref.index = [ dt.datetime.strftime(d,format="%Y-%m-%d %H:%M:%S") for d in curr_data_ref.index ]
        curr_data_ref.index.name = "datetime"
        curr_data_ref.to_csv( curr_output_path + "/" + var['output_name']['reference'] )

        del [curr_data_ref]

In [ ]:
### BUILD THE CASE STUDY GRID

start_metadata = "/media/windows/projects/bias_correction/data/meteo/cmcc_cm/observations/Stations_for_gis.xlsx"

tmp_file = pd.read_excel(start_metadata,engine='openpyxl')

In [ ]:
tmp_file[['NAME']].iloc(0)[0].values[0].split(" ")[-2]

In [ ]:
tmp_file.iloc[0]

In [ ]:
# ID,East,North,Elev
ids = []
east = []
north = []
elevation = []

metadata_std = pd.DataFrame()

for el in tmp_file.index:
    curr_id = tmp_file['NAME'].iloc[el].split(" ")[-2]
    if curr_id == "TX":
        if tmp_file['NAME'].iloc[el][0:2] == "SA":
            curr_id = "TX_SA"
        else:
            curr_id = "TX_D"
    ids.append( curr_id )

    east.append( tmp_file['LONGITUDE'].iloc[el] )
    north.append( tmp_file['LATITUDE'].iloc[el] )
    elevation.append( tmp_file['ELEVATION (m)'].iloc[el] )

metadata_std['ID'] = ids
metadata_std['East'] = east
metadata_std['North'] = north
metadata_std['Elev'] = elevation

metadata_std.set_index('ID', inplace=True)
metadata_std.sort_index(ascending=True, inplace=True)

metadata_std.to_csv( output_path + "USA_grid.csv" )